# YouTube Comment Cleaning and Exploration

Takes data from a CSV file and cleans the data to isolate comments of interest

In [35]:
import re
import pandas as pd
from collections import Counter
from fugashi import Tagger
from pathlib import Path

In [2]:
input_path = Path('Resources/youtube_comments.csv')
comments_df = pd.read_csv(input_path, index_col=0)

comments_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type
0,mwamjapan,Jb6Zlg30rgk,10,This season is going to be a masterpiece <3,2023-04-16T15:39:10Z,top-level
1,mwamjapan,Jb6Zlg30rgk,10,Every season is masterpiece 🔥🔥🔥,2023-04-17T16:41:52Z,reply
2,mwamjapan,Jb6Zlg30rgk,10,@HM cry about it,2023-04-17T16:15:19Z,reply
3,mwamjapan,Jb6Zlg30rgk,10,@HM dude demon slayer has no story but it has ...,2023-04-17T15:44:29Z,reply
4,mwamjapan,Jb6Zlg30rgk,10,@HM 🙂🙃😒😒😒,2023-04-17T13:44:05Z,reply
5,mwamjapan,Jb6Zlg30rgk,10,Honestamente no entiendo porqué esta canción r...,2023-04-17T03:58:25Z,top-level
6,mwamjapan,Jb6Zlg30rgk,10,​@みっくん𓈒𓂂◌𝙼𝙸𝚈𝚄 llora pues,2023-04-17T17:15:35Z,reply
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ！,2023-04-17T14:53:19Z,reply
9,mwamjapan,Jb6Zlg30rgk,10,@みっくん𓈒𓂂◌𝙼𝙸𝚈𝚄 ニワカほどそう言うよね！,2023-04-17T13:11:13Z,reply


In [3]:
# RE patterns needed for JP text
hiragana = r'\u3041-\u3096'
katakana = r'\u30A1-\u30F6'
kanji = r'\u3006\u4E00-\u9FFF'

# Matches to test for existence of text that uses a Japanese script
jp_text = rf'[{hiragana}{katakana}{kanji}]'

# Accepted characters
accepted_char = rf'[a-zA-Z0-9!?()\u30FC\u3005「」、。{hiragana}{katakana}{kanji}]'

In [4]:
# Remove @username from replies
# Done before rest of cleaning to ensure a user using a Japanese script doesn't pass filter
comments_df['text'] = comments_df['text'].str.replace(r'@\S+\s', '', regex=True)

In [5]:
# Filter out comments that don't have any Japanese characters
has_jp_char = comments_df.loc[comments_df['text'].str.contains(jp_text, regex=True)]

has_jp_char.sample(10)

,channel,video_id,category_id,text,date_published,comment_type
100,緑仙 / Ryushen,Sis3y7l8G98,24,オープニング良すぎでしょ,2023-04-10T04:13:00Z,top-level
77,細川バレンタイン / 前向き教室,wtbVhUgMS4M,17,当たり前なんだろけど本当に努力してるんだろうなと思う！\n本人もボクシング舐めてるわけではな...,2023-04-09T10:50:59Z,top-level
66,伊豆のぬし釣り,CTKRto5QFZo,19,伊豆のぬしずりさんも、しょこたんも最高です✨\nキャンプっていいなぁ♡ってこの動画見て本当思...,2023-04-10T11:30:43Z,top-level
71,緑仙 / Ryushen,Sis3y7l8G98,24,マジでまたやってくれw\n初めからずっと見てて全く飽きなくて楽しすぎた!\nみんなお疲れ様でした！,2023-04-09T11:03:51Z,top-level
8,伊豆のぬし釣り,CTKRto5QFZo,19,なんだかこっちまでキャンプに初めて行った時の気持ちに帰れるような神回でした🥺キャンプの楽しさ...,2023-04-11T03:42:40Z,top-level
43,BAD HOP,0yCXvGj0l-8,10,ダル,2023-04-14T02:53:59Z,reply
66,タイピー日記/taipi,Nu32a22KGWc,22,丸い猫ちゃん❤甘えん坊で可愛い過ぎ😍💕,2023-04-14T03:19:11Z,top-level
59,令和の虎CHANNEL,pT7AdB_uD_g,26,14:23\nここで同意したら終わりなのを分かって敢えて否定で構える所が立ち回りに抜け目無さ...,2023-04-16T19:44:33Z,top-level
123,wakatte.tv,u6K2TfadIqg,24,あるある,2023-04-11T21:18:34Z,reply
51,KAI Channel / 朝倉海,PKiHsPvavgA,24,めちゃ楽しみです！,2023-04-12T15:52:39Z,top-level


In [6]:
# Remove other unnecessary characters - 
def clean_text(text):
    
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'https?://[a-zA-Z0-9.-]*[/?[a-zA-Z0-9.-_]*]*', '', text)
    text = re.sub(r'笑+', '笑', text)
    
    filtered_text = ''
    
    for char in text:
        if re.match(accepted_char, char):
            
            filtered_text += char
            
    return filtered_text

In [7]:
# Clean
cleaned_df = has_jp_char.copy()
cleaned_df['text'] = cleaned_df['text'].apply(clean_text)

cleaned_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ,2023-04-17T14:53:19Z,reply
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど、声の相性が良くて特に重なった時に何とも言えない...,2023-04-17T00:54:46Z,top-level
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで、作中にはいろんな絆ノ奇跡がありますよね。特に刀鍛冶の里編のラストは「絆ノ奇...,2023-04-17T13:48:13Z,reply
46,mwamjapan,Jb6Zlg30rgk,10,「我が命果てようとも繋いでいこう」自分が志半ばで死んでも繋いだ命がきっと果たしてくれる、と二...,2023-04-17T13:37:24Z,reply
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました。,2023-04-17T13:07:22Z,reply
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には、色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply


In [8]:
# Check size of DF
cleaned_df.shape

(11297, 6)

In [9]:
# Check for null values
cleaned_df.isna().sum()

channel           0
video_id          0
category_id       0
text              0
date_published    0
comment_type      0
dtype: int64

In [10]:
# Check unique values in each
cleaned_df.nunique()

channel              82
video_id             92
category_id          12
text              10670
date_published    11104
comment_type          2
dtype: int64

In [11]:
# Check amounts of top-level vs. replies for comments
cleaned_df['comment_type'].value_counts()

top-level    8320
reply        2977
Name: comment_type, dtype: int64

In [12]:
# Create instance of fugashi tagger
tagger = Tagger()

In [43]:
# Tokenize text
analysis_df = cleaned_df.copy()
analysis_df['tokens'] = analysis_df['text'].apply(lambda x: tagger(x))

analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,"[た, か, が, OP, で, 世界, 観, が, 壊れる, アニメ, じ, ゃないわはげたこ]"
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ,2023-04-17T14:53:19Z,reply,"[悩め, ば, 、, イイ]"
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,"[ニワカ, ほど, そう, 言う, よ, ね]"
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,"[珍しく, ショート, 動画, が, 出, て, ます, ね, 最高, すぎる]"
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,"[めっちゃ, 同意]"
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど、声の相性が良くて特に重なった時に何とも言えない...,2023-04-17T00:54:46Z,top-level,"[milet, と, の, コラボ, に, は, 最初, びっくり, し, た, けど, 、..."
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで、作中にはいろんな絆ノ奇跡がありますよね。特に刀鍛冶の里編のラストは「絆ノ奇...,2023-04-17T13:48:13Z,reply,"[全く, 仰る, とおり, で, 、, 作中, に, は, いろんな, 絆, ノ, 奇跡, ..."
46,mwamjapan,Jb6Zlg30rgk,10,「我が命果てようとも繋いでいこう」自分が志半ばで死んでも繋いだ命がきっと果たしてくれる、と二...,2023-04-17T13:37:24Z,reply,"[「, 我が, 命, 果てよう, と, も, 繋い, で, いこう, 」, 自分, が, 志..."
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました。,2023-04-17T13:07:22Z,reply,"[我が, 命, 果てよう, とも, に, は, 無, 一郎, を, 思い浮かべ, まし, た..."
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には、色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,"[鬼, 滅, の, 刃, に, は, 、, 色んな, 絆, ノ, 奇跡, が, あり, ます..."


In [45]:
# Remove instances of punctuation
# Done now to allow fugashi to have additional help with defining boundaries
punct = '\.,\'\"。、「」（）～~\(\)'

def remove_punctuation(text):
    
    filtered_text = []
    
    for word in text:
        if str(word) not in punct:
            filtered_text.append(word)
            
    return filtered_text

analysis_df['tokens'] = analysis_df['tokens'].apply(remove_punctuation)

analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,"[た, か, が, OP, で, 世界, 観, が, 壊れる, アニメ, じ, ゃないわはげたこ]"
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ,2023-04-17T14:53:19Z,reply,"[悩め, ば, イイ]"
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,"[ニワカ, ほど, そう, 言う, よ, ね]"
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,"[珍しく, ショート, 動画, が, 出, て, ます, ね, 最高, すぎる]"
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,"[めっちゃ, 同意]"
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど、声の相性が良くて特に重なった時に何とも言えない...,2023-04-17T00:54:46Z,top-level,"[milet, と, の, コラボ, に, は, 最初, びっくり, し, た, けど, 声..."
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで、作中にはいろんな絆ノ奇跡がありますよね。特に刀鍛冶の里編のラストは「絆ノ奇...,2023-04-17T13:48:13Z,reply,"[全く, 仰る, とおり, で, 作中, に, は, いろんな, 絆, ノ, 奇跡, が, ..."
46,mwamjapan,Jb6Zlg30rgk,10,「我が命果てようとも繋いでいこう」自分が志半ばで死んでも繋いだ命がきっと果たしてくれる、と二...,2023-04-17T13:37:24Z,reply,"[我が, 命, 果てよう, と, も, 繋い, で, いこう, 自分, が, 志半ば, で,..."
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました。,2023-04-17T13:07:22Z,reply,"[我が, 命, 果てよう, とも, に, は, 無, 一郎, を, 思い浮かべ, まし, た]"
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には、色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,"[鬼, 滅, の, 刃, に, は, 色んな, 絆, ノ, 奇跡, が, あり, ます, ね]"


In [53]:
test = analysis_df.iloc[2]['tokens']
for word in test:
    print(word, word.feature.pos1)

ニワカ 名詞
ほど 助動詞
そう 動詞
言う 助詞
よ 接尾辞
ね 名詞


In [47]:
# Currently consider non-particles or auxiliary/inflecting verbs
def filtered_words(text):
    
    filtered_text = []
    
    for word in text:
        if word.feature.pos1 != '助詞' and word.feature.pos1 != '助動詞':
            
            filtered_text.append(word)
    
    return filtered_text

analysis_df['filtered_tokens'] = analysis_df['tokens'].apply(filtered_words)
analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens,filtered_tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,"[た, か, が, OP, で, 世界, 観, が, 壊れる, アニメ, じ, ゃないわはげたこ]","[た, か, OP, 世界, アニメ, じ, ゃないわはげたこ]"
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ,2023-04-17T14:53:19Z,reply,"[悩め, ば, イイ]","[悩め, ば, イイ]"
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,"[ニワカ, ほど, そう, 言う, よ, ね]","[ニワカ, そう, よ, ね]"
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,"[珍しく, ショート, 動画, が, 出, て, ます, ね, 最高, すぎる]","[ショート, が, 出, て, ます, ね, 最高]"
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,"[めっちゃ, 同意]",[]
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど、声の相性が良くて特に重なった時に何とも言えない...,2023-04-17T00:54:46Z,top-level,"[milet, と, の, コラボ, に, は, 最初, びっくり, し, た, けど, 声...","[milet, と, の, に, びっくり, し, た, けど, 声, の, 相性, 良く,..."
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで、作中にはいろんな絆ノ奇跡がありますよね。特に刀鍛冶の里編のラストは「絆ノ奇...,2023-04-17T13:48:13Z,reply,"[全く, 仰る, とおり, で, 作中, に, は, いろんな, 絆, ノ, 奇跡, が, ...","[全く, 仰る, とおり, で, は, いろんな, ノ, 奇跡, が, あり, ます, よ,..."
46,mwamjapan,Jb6Zlg30rgk,10,「我が命果てようとも繋いでいこう」自分が志半ばで死んでも繋いだ命がきっと果たしてくれる、と二...,2023-04-17T13:37:24Z,reply,"[我が, 命, 果てよう, と, も, 繋い, で, いこう, 自分, が, 志半ば, で,...","[我が, 命, 果てよう, と, も, 繋い, で, いこう, 自分, が, 志半ば, 死ん..."
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました。,2023-04-17T13:07:22Z,reply,"[我が, 命, 果てよう, とも, に, は, 無, 一郎, を, 思い浮かべ, まし, た]","[我が, 命, に, は, 無, 一郎, を, 思い浮かべ, まし, た]"
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には、色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,"[鬼, 滅, の, 刃, に, は, 色んな, 絆, ノ, 奇跡, が, あり, ます, ね]","[鬼, 滅, の, 刃, は, 色んな, あり, ます, ね]"


In [ ]:
# Looks like filtering particles out a bit more complicated...